In [1]:
import sys
sys.path.append('../')
from utils import preprocess, Tokenizer4keras, create_fastText_model

import numpy as np
import h5py

import pandas as pd

from collections import defaultdict
import string

from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical

np.random.seed(1234)

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv('./../data/train_feature.csv')
df_test = pd.read_csv('./../data/test_feature.csv')
df_train_texts = df_train.text.values
df_test_tests = df_test.text.values

author2class = {'EAP': 0, 'HPL' : 1, 'MWS' : 2}
class2author = ['EAP', 'HPL', 'MWS']
y = np.array([author2class[a] for a in df_train.author])
y = to_categorical(y)

In [3]:
# preprocessin parameters
n_gram_max = 1
min_count = 1
maxlen = 512

tokenizer = Tokenizer4keras(maxlen=maxlen, min_count=min_count, n_gram_max=n_gram_max, lower=False, single=False, add_ngram_first=True)
x = tokenizer.fit_transform(df_texts=df_train_texts)
x_test = tokenizer.transofrm(df_test_tests)

In [4]:
input_dim = np.max(x) + 1

# for next training
predict_prob_features = np.zeros((len(df_train), 3))
predict_prob_features_test = np.zeros((len(df_test), 3))

# training parameters
seed = 7
num_split = 5
epochs = 60

ite = 0
sum_loss = 0.
losses = []

kf = KFold(n_splits=num_split, random_state=seed, shuffle=True)
for train_index, val_index in kf.split(x):
    ite += 1
    print('{}/{}: #Trains: {}, #Val: {}'.format(ite, num_split, len(train_index), len(val_index)), end=' ')
    
    x_train = x[train_index]
    x_val = x[val_index]

    y_train, y_val = y[train_index], y[val_index]

    print(x_train.shape, x_val.shape, x_test.shape)
    
    model = create_fastText_model(input_dim, embedding_dim=10, optimizer='adam')
    
    checkpointer = ModelCheckpoint(filepath='./../fasttext_weights/weights.hdf5', verbose=0, save_best_only=True)

    hist = model.fit(x_train, y_train,
                     batch_size=16,
                     validation_data=(x_val, y_val),
                     epochs=epochs,
                     callbacks=[EarlyStopping(patience=4, monitor='val_loss'), checkpointer]
                    )

    # load best weights
    model.load_weights('./../fasttext_weights/weights.hdf5')
    y_pred = model.predict_proba(x_val)
    l = log_loss(y_pred=y_pred, y_true=np.nonzero(y_val)[1])
    losses.append(l)
    sum_loss += l
    
    print('valLoss: {}'.format(sum_loss/ite))

    # save features
    predict_prob_features[val_index] = y_pred
    predict_prob_features_test += model.predict_proba(x_test)

1/5: #Trains: 15663, #Val: 3916 (15663, 512) (3916, 512) (8392, 512)
Train on 15663 samples, validate on 3916 samples
Epoch 1/60
15663/15663 [==============================] - 12s - loss: 1.0852 - acc: 0.4037 - val_loss: 1.0803 - val_acc: 0.4035
Epoch 2/60
15663/15663 [==============================] - 11s - loss: 1.0709 - acc: 0.4042 - val_loss: 1.0630 - val_acc: 0.4063
Epoch 3/60
15663/15663 [==============================] - 8s - loss: 1.0388 - acc: 0.4340 - val_loss: 1.0205 - val_acc: 0.4385
Epoch 4/60
15663/15663 [==============================] - 7s - loss: 0.9851 - acc: 0.5372 - val_loss: 0.9657 - val_acc: 0.5143
Epoch 5/60
15663/15663 [==============================] - 7s - loss: 0.9191 - acc: 0.6439 - val_loss: 0.9076 - val_acc: 0.5692
Epoch 6/60
15663/15663 [==============================] - 7s - loss: 0.8512 - acc: 0.7043 - val_loss: 0.8453 - val_acc: 0.6701
Epoch 7/60
15663/15663 [==============================] - 8s - loss: 0.7873 - acc: 0.7395 - val_loss: 0.7933 - val_acc

15663/15663 [==============================] - 7s - loss: 0.3102 - acc: 0.9056 - val_loss: 0.4466 - val_acc: 0.8309
Epoch 24/60
15663/15663 [==============================] - 7s - loss: 0.2957 - acc: 0.9116 - val_loss: 0.4387 - val_acc: 0.8299
Epoch 25/60
15663/15663 [==============================] - 8s - loss: 0.2814 - acc: 0.9157 - val_loss: 0.4304 - val_acc: 0.8345
Epoch 26/60
15663/15663 [==============================] - 8s - loss: 0.2681 - acc: 0.9208 - val_loss: 0.4226 - val_acc: 0.8368
Epoch 27/60
15663/15663 [==============================] - 6s - loss: 0.2564 - acc: 0.9250 - val_loss: 0.4206 - val_acc: 0.8376
Epoch 28/60
15663/15663 [==============================] - 7s - loss: 0.2448 - acc: 0.9280 - val_loss: 0.4106 - val_acc: 0.8399
Epoch 29/60
15663/15663 [==============================] - 7s - loss: 0.2345 - acc: 0.9323 - val_loss: 0.4051 - val_acc: 0.8424
Epoch 30/60
15663/15663 [==============================] - 7s - loss: 0.2243 - acc: 0.9349 - val_loss: 0.4053 - val_

7840/8392 [===========================>..] - ETA: 0s

In [5]:
for a, c in author2class.items():
    df_train['{}_fasttext_unigram'.format(a)] = predict_prob_features[:, c]
    df_test['{}_fasttext_unigram'.format(a)] = predict_prob_features_test[:, c]/num_split

In [7]:
df_train.to_csv('./../data/train_feature.csv', index=False)
df_test.to_csv('./../data/test_feature.csv', index=False)